In [61]:
import os

from sklearn.metrics import cohen_kappa_score
import itertools

from collections import defaultdict

import re

from typing import Dict, List

import numpy as np
import math

In [62]:
DATA_PATH = os.path.join('..', 'data_2', 'Keep')

In [63]:
def read_data():
    doc_annotations = {}
    text_lengths = {}

    authors_set = set()

    for author_folder in os.listdir(DATA_PATH):
        authors_set.add(author_folder)
        full_path = os.path.join(DATA_PATH, author_folder)
        ann_files = list(filter(lambda x: x.endswith('.ann'), os.listdir(full_path)))
        for filename in ann_files:
            doc_name = os.path.splitext(filename)[0]
            if doc_name not in doc_annotations.keys():
                doc_annotations[doc_name] = {}

            if author_folder in doc_annotations[doc_name].keys():
                raise Exception(f'Author "{author_folder}" has duplicated annotation for document "{doc_name}"')

            with open(os.path.join(full_path, filename), 'r') as file_handler:
                doc_annotations[doc_name][author_folder] = file_handler.read()

            if doc_name not in text_lengths.keys():
                with open(os.path.join(full_path, f'{doc_name}.txt'), 'r') as txt_file_handler:
                    text_lengths[doc_name] = len(txt_file_handler.read())

    return doc_annotations, text_lengths, list(sorted(authors_set))

In [64]:
doc_annotations, text_lengths, authors_set = read_data()

In [65]:
valid_annotations = {
    k: v for k, v in doc_annotations.items()
    if len(v.keys()) > 1
}

In [66]:
class Annotation:
    def __init__(self, key: str, entity: str, start_pos: int, end_pos: int):
        self.key = key
        self.entity = entity
        self.start_pos = start_pos
        self.end_pos = end_pos

    def __str__(self):
        return f'<{self.key}-{self.entity}-[{self.start_pos}:{self.end_pos}]>'

In [67]:
invalid_labels = ['DuplicatePage', 'TranscriptionError_Document']

class DocumentAnnotation:
    def __init__(self, doc_key: str, annotations_str: str, text_length: int, labels_to_use: List[str]):
        self.doc_key = doc_key
        self.text_length = text_length
        self.is_valid=True

        self.annotations = self._parse_annotations(annotations_str, labels_to_use)

    def _parse_annotations(self, annotations_str: str, labels_to_use: List[str]):
        result = []
        annotations = annotations_str.split('\n')
        for annotation in annotations:
            annotation_parts = annotation.split('\t')
            if not re.search('([T]{1}[0-9]+)', annotation_parts[0]): continue
            if len(annotation_parts) < 2: print(annotation_parts)

            split_annotation = annotation_parts[1].split(' ')
            label = split_annotation[0]

            if label in invalid_labels:
                self.is_valid=False
                continue

            if label not in labels_to_use:
                continue

            current_annotations_parts = ' '.join(split_annotation[1:]).split(';')

            start = int(current_annotations_parts[0].split(' ')[0])
            end = int(current_annotations_parts[-1].split(' ')[-1])

            result.append(
                Annotation(
                    key=annotation_parts[0],
                    entity=label,
                    start_pos=start,
                    end_pos=end))

        return result

In [68]:
def parse_annotations(valid_annotations, text_lengths: Dict[str, int], labels_to_use: List[str]):
    parsed_annotations = {
        doc_key: {
            author: DocumentAnnotation(doc_key, annotations, text_lengths[doc_key], labels_to_use)
            for author, annotations in annotations_per_author.items()
        }
        for doc_key, annotations_per_author in valid_annotations.items()
    }

    # Remove invalid annotations
    parsed_annotations = {
        doc_key: {
            author: doc_annotation
            for author, doc_annotation in annotations_per_author.items()
            if doc_annotation.is_valid
        }
        for doc_key, annotations_per_author in parsed_annotations.items()
    }

    # Remove documents where we are left with only one annotation
    parsed_annotations = {
        doc_key: annotations_per_author
        for doc_key, annotations_per_author in parsed_annotations.items()
        if len(annotations_per_author.keys()) > 1
    }

    return parsed_annotations

In [69]:
def annotations_overlap(annotation1: Annotation, annotation2: Annotation, offset_chars: int, match_entity: bool) -> bool:
    # if the two annotations do not even overlap with one character, we return false
    if (annotation1.start_pos > annotation2.end_pos or
        annotation1.end_pos < annotation2.start_pos):
        return False

    if match_entity and (annotation1.entity != annotation2.entity):
        return False

    out_of_boundary_chars = abs(annotation1.start_pos - annotation2.start_pos) + abs(annotation1.end_pos - annotation2.end_pos)

    result = out_of_boundary_chars <= offset_chars
    return result

In [70]:
def print_mapped_annotations(mapped_annotations: dict):
    for ann1, ann2 in mapped_annotations.items():
        print(f'{ann1.key} <{ann1.start_pos}-{ann1.end_pos}>  ---', end='')
        if ann2 is None:
            print('NONE')
        else:
            print(f'{ann2.key} <{ann2.start_pos}-{ann2.end_pos}>')

In [71]:
def get_overlapping_annotation(annotation_to_compare: Annotation, annotations: List[Annotation], keys_to_skip: List[str], offset_chars: int, match_entity: bool) -> Annotation:
    overlaps = []
    for annotation2 in annotations:

        if annotation_to_compare.start_pos > annotation2.end_pos:
            continue

        if annotation2.key in keys_to_skip:
            continue

        if annotations_overlap(annotation_to_compare, annotation2, offset_chars, match_entity):
            overlaps.append(annotation2)

    if len(overlaps) == 0:
        return None

    for overlap in overlaps:
        if overlap.entity == annotation_to_compare.entity:
            return overlap

    return overlaps[0]

In [72]:
def calculate_entity_overlap(doc_annotation1: DocumentAnnotation, doc_annotation2: DocumentAnnotation, offset_chars: int, debug: bool = False):
    assert (doc_annotation1.doc_key == doc_annotation2.doc_key)

    mapped_annotations = {}
    used_counter_annotations = set()
    empty_positions = [1 for _ in range(0, doc_annotation1.text_length)]
    for annotation in doc_annotation1.annotations:
        for i in range(annotation.start_pos, annotation.end_pos):
            empty_positions[i] = 0

    # Perform iteration using strict overlap matching
    for annotation in doc_annotation1.annotations:
        overlapping_annotation = get_overlapping_annotation(annotation, doc_annotation2.annotations, used_counter_annotations, offset_chars, match_entity=True)
        if overlapping_annotation is not None:
            used_counter_annotations.add(overlapping_annotation.key)
            mapped_annotations[annotation] = overlapping_annotation

    # Perform iteration using loose overlap matching
    for annotation in doc_annotation1.annotations:
        if annotation in mapped_annotations.keys():
            continue

        overlapping_annotation = get_overlapping_annotation(annotation, doc_annotation2.annotations, used_counter_annotations, offset_chars, match_entity=False)
        if overlapping_annotation is not None:
            used_counter_annotations.add(overlapping_annotation.key)

        mapped_annotations[annotation] = overlapping_annotation

    if debug:
        print_mapped_annotations(mapped_annotations)
    annotation_maps = [ x.entity for x in mapped_annotations.keys() ]
    counter_annotations = [ x.entity if x is not None else 'O' for x in mapped_annotations.values() ]

    for annotation2 in doc_annotation2.annotations:
        for i in range(annotation2.start_pos, annotation2.end_pos):
            empty_positions[i] = 0

        if annotation2.key in used_counter_annotations:
            continue

        annotation_maps.append('O')
        counter_annotations.append(annotation2.entity)

    free_positions = sum(empty_positions)
    for _ in range(free_positions):
        annotation_maps.append('O')
        counter_annotations.append('O')

    if annotation_maps == counter_annotations:
        result = 1
    else:
        result = cohen_kappa_score(annotation_maps, counter_annotations)

    return result

In [81]:
def create_comparison_matrix(parsed_annotations, offset_chars: int, authors_set: set):
    comparisons = {
        author_1 : {
            author_2: []
            for author_2 in authors_set
        } for author_1 in authors_set
    }

    for _, annotations in parsed_annotations.items():
        for author_1, author_2 in itertools.product(authors_set, authors_set):
            if author_1 in annotations.keys() and author_2 in annotations.keys():
                debug=False
                kappa_score = calculate_entity_overlap(annotations[author_1], annotations[author_2], offset_chars, debug=debug)

                comparisons[author_1][author_2].append(kappa_score)

        for author in authors_set:
            comparisons[author][author] = [1]

    return comparisons

In [82]:
def print_comparison_matrix(comparisons, authors_set: List[str], offset_chars: int, labels_to_use: List[str]):
    print (f'Results, offset characters: {offset_chars}, used labels: <{",".join(labels_to_use)}>')
    print('\t', end='')
    print('\t'.join(authors_set))
    for author_1 in authors_set:
        print(author_1, end='\t')
        for author_2 in authors_set:
            print(round(np.mean(comparisons[author_1][author_2]), 2), end='\t')

        print()

In [83]:
def run_comparison(valid_annotations, text_lengths: Dict[str, int], authors_set: List[str], offset_chars: int, labels_to_use: List[str]):
    parsed_annotations = parse_annotations(valid_annotations, text_lengths, labels_to_use)
    comparison_matrix = create_comparison_matrix(parsed_annotations, offset_chars, authors_set)
    print_comparison_matrix(comparison_matrix, authors_set, offset_chars, labels_to_use)

In [84]:
run_comparison(valid_annotations, text_lengths, authors_set, offset_chars=0, labels_to_use=['Person', 'Place', 'Organization'])

Results, offset characters: 0, used labels: <Person,Place,Organization>
	Bert	Emma	Jonas	Roos	Silja	Yolien
Bert	1.0	0.43	0.42	0.44	0.5	0.45	
Emma	0.43	1.0	0.47	0.63	0.35	0.47	
Jonas	0.42	0.47	1.0	0.48	0.47	0.38	
Roos	0.44	0.63	0.48	1.0	0.55	0.47	
Silja	0.5	0.35	0.47	0.55	1.0	0.48	
Yolien	0.45	0.47	0.38	0.47	0.48	1.0	


In [85]:
run_comparison(valid_annotations, text_lengths, authors_set, offset_chars=0, labels_to_use=['Person', 'Place', 'Organization', 'ProperName', 'Noteworthy'])

Results, offset characters: 0, used labels: <Person,Place,Organization,ProperName,Noteworthy>
	Bert	Emma	Jonas	Roos	Silja	Yolien
Bert	1.0	0.53	0.51	0.46	0.64	0.64	
Emma	0.53	1.0	0.5	0.69	0.5	0.52	
Jonas	0.51	0.5	1.0	0.55	0.49	0.43	
Roos	0.46	0.69	0.55	1.0	0.62	0.56	
Silja	0.64	0.5	0.49	0.62	1.0	0.48	
Yolien	0.64	0.52	0.43	0.56	0.48	1.0	


In [86]:
run_comparison(valid_annotations, text_lengths, authors_set, offset_chars=50, labels_to_use=['Person', 'Place', 'Organization'])

Results, offset characters: 50, used labels: <Person,Place,Organization>
	Bert	Emma	Jonas	Roos	Silja	Yolien
Bert	1.0	0.77	0.74	0.65	0.81	0.89	
Emma	0.77	1.0	0.85	0.89	0.85	0.88	
Jonas	0.74	0.85	1.0	0.78	0.81	0.81	
Roos	0.65	0.89	0.78	1.0	0.85	0.85	
Silja	0.81	0.85	0.81	0.85	1.0	0.79	
Yolien	0.89	0.88	0.81	0.85	0.79	1.0	


In [87]:
run_comparison(valid_annotations, text_lengths, authors_set, offset_chars=500, labels_to_use=['Person', 'Place', 'Organization'])

Results, offset characters: 500, used labels: <Person,Place,Organization>
	Bert	Emma	Jonas	Roos	Silja	Yolien
Bert	1.0	0.78	0.74	0.65	0.81	0.89	
Emma	0.78	1.0	0.87	0.9	0.85	0.88	
Jonas	0.74	0.87	1.0	0.82	0.83	0.86	
Roos	0.65	0.9	0.82	1.0	0.87	0.85	
Silja	0.81	0.85	0.83	0.87	1.0	0.8	
Yolien	0.89	0.88	0.86	0.85	0.8	1.0	


In [88]:
run_comparison(valid_annotations, text_lengths, authors_set, offset_chars=500, labels_to_use=['Person', 'Place', 'Organization', 'ProperName', 'Noteworthy'])

Results, offset characters: 500, used labels: <Person,Place,Organization,ProperName,Noteworthy>
	Bert	Emma	Jonas	Roos	Silja	Yolien
Bert	1.0	0.77	0.71	0.66	0.83	0.91	
Emma	0.77	1.0	0.77	0.89	0.88	0.88	
Jonas	0.71	0.77	1.0	0.76	0.77	0.78	
Roos	0.66	0.89	0.76	1.0	0.84	0.84	
Silja	0.83	0.88	0.77	0.84	1.0	0.8	
Yolien	0.91	0.88	0.78	0.84	0.8	1.0	
